# Pump Locker Test - Dual HiScroll12 Control System

This notebook provides control and monitoring capabilities for two HiScroll12 vacuum pumps with TPG366 pressure sensors using the current hardware versions.

## Features:
- Control for 2 HiScroll12 vacuum pumps (current version)
- Monitoring for TPG366 pressure sensors (current version)
- Shared logging system using loguru
- Cancelable continuous monitoring
- Real-time data visualization
- Simplified and optimized for current hardware

## 1. Import Required Libraries and Setup

In [ ]:
import sys
import threading
import time
from datetime import datetime
import asyncio
from typing import Dict, Optional, Any
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import pickle

from loguru import logger
import os
from pathlib import Path

# Add path to src modules
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))

# Import current device modules
from devices.pfeiffer.hiscroll12.hiscroll12 import HiScroll12
from devices.pfeiffer.tpg366.tpg366 import TPG366

## 2. Setup Shared Logging System

In [ ]:
# Get the repository root and create shared logs directory
repo_root = Path(os.getcwd()).parent.parent
log_dir = repo_root / "debugging" / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

# Create shared log file for all devices
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
shared_log_file = log_dir / f"pump_locker_test_{timestamp}.log"

# Configure shared logger
logger.remove()  # Remove default logger

# Add console logger with INFO level
logger.add(sys.stderr, level="INFO", format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}")

# Add shared file logger with DEBUG level
logger.add(
    str(shared_log_file),
    level="DEBUG",
    format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level} | {name}:{function}:{line} | {message}",
    rotation="1 day",
    retention="30 days",
    compression="zip"
)

logger.info("Pump Locker Test system initialized with shared logging")
print(f"Repository root: {repo_root}")
print(f"Shared logs will be saved to: {shared_log_file}")

## 3. Simplified Pump Controller Class

In [ ]:
class PumpLockerController:
    """Simplified controller class for managing two HiScroll12 pumps and TPG366 pressure sensors"""
    
    def __init__(self):
        self.pump1: Optional[HiScroll12] = None
        self.pump2: Optional[HiScroll12] = None
        self.pressure_controller: Optional[TPG366] = None
        
        # Monitoring control
        self.monitoring_active = False
        self.monitoring_thread = None
        self.monitoring_stop_event = threading.Event()
        
        # Data storage
        self.monitoring_data = []
        
        # Shared thread lock for all devices
        self.shared_lock = threading.Lock()
        
        logger.info("PumpLockerController initialized")
    
    def connect_pumps(self, pump1_com: str, pump1_addr: int, pump2_com: str, pump2_addr: int):
        """Connect to both HiScroll12 pumps with shared logger"""
        try:
            logger.info(f"Connecting to Pump 1 on {pump1_com}, address {pump1_addr}")
            self.pump1 = HiScroll12(
                device_id="pump_1",
                port=pump1_com,
                device_address=pump1_addr,
                logger=logger,  # Pass shared logger
                thread_lock=self.shared_lock  # Use shared lock
            )
            self.pump1.connect()
            
            logger.info(f"Connecting to Pump 2 on {pump2_com}, address {pump2_addr}")
            self.pump2 = HiScroll12(
                device_id="pump_2",
                port=pump2_com,
                device_address=pump2_addr,
                logger=logger,  # Pass shared logger
                thread_lock=self.shared_lock  # Use shared lock
            )
            self.pump2.connect()
            
            logger.success("Both HiScroll12 pumps connected successfully")
            return True
            
        except Exception as e:
            logger.error(f"Failed to connect pumps: {e}")
            return False
    
    def connect_pressure_controller(self, com_port: str, address: int):
        """Connect to TPG366 pressure controller with shared logger"""
        try:
            logger.info(f"Connecting to TPG366 pressure controller on {com_port}, address {address}")
            self.pressure_controller = TPG366(
                device_id="tpg366_pressure",
                port=com_port,
                device_address=address,
                logger=logger,  # Pass shared logger
                thread_lock=self.shared_lock  # Use shared lock
            )
            self.pressure_controller.connect()
            
            logger.success("TPG366 pressure controller connected successfully")
            return True
            
        except Exception as e:
            logger.error(f"Failed to connect TPG366 pressure controller: {e}")
            return False
    
    def disconnect_all(self):
        """Safely disconnect all devices"""
        logger.info("Disconnecting all devices...")
        
        # Stop monitoring first
        self.stop_monitoring()
        
        # Disconnect pumps
        if self.pump1:
            try:
                self.pump1.disconnect()
                logger.info("Pump 1 disconnected")
            except Exception as e:
                logger.error(f"Error disconnecting Pump 1: {e}")
        
        if self.pump2:
            try:
                self.pump2.disconnect()
                logger.info("Pump 2 disconnected")
            except Exception as e:
                logger.error(f"Error disconnecting Pump 2: {e}")
        
        # Disconnect pressure controller
        if self.pressure_controller:
            try:
                self.pressure_controller.disconnect()
                logger.info("TPG366 pressure controller disconnected")
            except Exception as e:
                logger.error(f"Error disconnecting TPG366 pressure controller: {e}")
        
        logger.success("All devices disconnected")

# Create global controller instance
controller = PumpLockerController()

## 4. Device Monitoring Functions

In [ ]:
def monitor_hiscroll12_pump(pump: HiScroll12, pump_id: str) -> Dict[str, Any]:
    """
    Monitor a HiScroll12 pump and return its status parameters.
    
    Parameters:
    - pump: HiScroll12 pump instance
    - pump_id: Identifier string for the pump
    
    Returns:
    - Dictionary containing pump parameters
    """
    
    try:
        logger.debug(f"Monitoring {pump_id}...")
        
        # Get actual pump data using current API
        pump_data = {
            'timestamp': datetime.now(),
            'pump_id': pump_id,
            'rotation_speed_rpm': pump.get_actual_rotation_speed_rpm(),
            'rotation_speed_hz': pump.get_actual_rotation_speed_hz(),
            'pump_enabled': pump.get_pump_enable(),
            'standby_mode': pump.get_standby_mode(),
            'drive_power': pump.get_drive_power(),
            'drive_current': pump.get_drive_current(),
            'drive_voltage': pump.get_drive_voltage(),
            'temp_motor': pump.get_temp_motor(),
            'temp_electronics': pump.get_temp_electronics(),
            'temp_power_stage': pump.get_temp_power_stage(),
            'operating_time_pump': pump.get_pump_operating_time(),
            'operating_time_electronics': pump.get_electronics_operating_time(),
            'status': 'OK'
        }
        
        # Log key parameters
        logger.info(f"{pump_id} - RPM: {pump_data['rotation_speed_rpm']}, Enabled: {pump_data['pump_enabled']}, Power: {pump_data['drive_power']}W")
        logger.debug(f"{pump_id} monitoring completed successfully")
        return pump_data
        
    except Exception as e:
        logger.error(f"Error monitoring {pump_id}: {e}")
        return {
            'timestamp': datetime.now(),
            'pump_id': pump_id,
            'error': str(e),
            'status': 'ERROR'
        }

def monitor_tpg366_pressure(pressure_controller: TPG366, sensor_id: str, channel: int) -> Dict[str, Any]:
    """
    Monitor pressure from TPG366 sensor channel.
    
    Parameters:
    - pressure_controller: TPG366 pressure controller instance
    - sensor_id: Identifier string for the sensor
    - channel: Sensor channel to read from (1-6)
    
    Returns:
    - Dictionary containing pressure data
    """
    
    try:
        logger.debug(f"Reading pressure from {sensor_id}, channel {channel}...")
        
        # Get actual pressure reading using current API
        pressure_value = pressure_controller.read_pressure_value(channel)
        
        pressure_data = {
            'timestamp': datetime.now(),
            'sensor_id': sensor_id,
            'channel': channel,
            'pressure_value': pressure_value,
            'status': 'OK'
        }
        
        logger.debug(f"{sensor_id} pressure reading: {pressure_value}")
        return pressure_data
        
    except Exception as e:
        logger.error(f"Error reading pressure from {sensor_id}: {e}")
        return {
            'timestamp': datetime.now(),
            'sensor_id': sensor_id,
            'channel': channel,
            'pressure_value': None,
            'error': str(e),
            'status': 'ERROR'
        }

print("Monitoring functions defined successfully")

## 5. Continuous Monitoring System

In [ ]:
def continuous_monitoring_worker(controller: PumpLockerController, interval: float = 5.0):
    """
    Worker function for continuous monitoring that runs in a separate thread.
    """
    logger.info(f"Starting continuous monitoring with {interval}s interval")
    
    while not controller.monitoring_stop_event.is_set():
        try:
            monitoring_cycle_data = {
                'cycle_timestamp': datetime.now(),
                'pump1_data': None,
                'pump2_data': None,
                'sensor1_data': None,
                'sensor2_data': None
            }
            
            # Monitor pumps if connected
            if controller.pump1:
                monitoring_cycle_data['pump1_data'] = monitor_hiscroll12_pump(controller.pump1, "Pump_1")
            
            if controller.pump2:
                monitoring_cycle_data['pump2_data'] = monitor_hiscroll12_pump(controller.pump2, "Pump_2")
            
            # Monitor pressure sensors if controller is connected
            if controller.pressure_controller:
                # Read from sensor channels 1 and 2
                monitoring_cycle_data['sensor1_data'] = monitor_tpg366_pressure(
                    controller.pressure_controller, "Sensor_1", channel=1
                )
                
                monitoring_cycle_data['sensor2_data'] = monitor_tpg366_pressure(
                    controller.pressure_controller, "Sensor_2", channel=2
                )
            
            # Store data
            controller.monitoring_data.append(monitoring_cycle_data)
            
            # Log summary with pressure values if available
            log_msg = f"Monitoring cycle completed at {monitoring_cycle_data['cycle_timestamp']}"
            if monitoring_cycle_data['sensor1_data'] and monitoring_cycle_data['sensor2_data']:
                p1 = monitoring_cycle_data['sensor1_data'].get('pressure_value')
                p2 = monitoring_cycle_data['sensor2_data'].get('pressure_value')
                log_msg += f" | Pressures: S1={p1}, S2={p2}"
            logger.info(log_msg)
            
            # Wait for next cycle or stop signal
            if controller.monitoring_stop_event.wait(timeout=interval):
                break  # Stop event was set
                
        except Exception as e:
            logger.error(f"Error in monitoring cycle: {e}")
            # Continue monitoring even if one cycle fails
            time.sleep(interval)
    
    logger.info("Continuous monitoring stopped")

def start_monitoring(controller: PumpLockerController, interval: float = 5.0):
    """Start continuous monitoring in a separate thread."""
    if controller.monitoring_active:
        logger.warning("Monitoring is already active")
        return False
    
    # Reset stop event and clear old data
    controller.monitoring_stop_event.clear()
    controller.monitoring_data.clear()
    
    # Start monitoring thread
    controller.monitoring_thread = threading.Thread(
        target=continuous_monitoring_worker,
        args=(controller, interval),
        daemon=True
    )
    
    controller.monitoring_thread.start()
    controller.monitoring_active = True
    
    logger.success(f"Continuous monitoring started with {interval}s interval")
    return True

def stop_monitoring(controller: PumpLockerController):
    """Stop continuous monitoring gracefully."""
    if not controller.monitoring_active:
        logger.info("Monitoring is not currently active")
        return
    
    logger.info("Stopping continuous monitoring...")
    
    # Signal the monitoring thread to stop
    controller.monitoring_stop_event.set()
    
    # Wait for thread to finish (with timeout)
    if controller.monitoring_thread and controller.monitoring_thread.is_alive():
        controller.monitoring_thread.join(timeout=10.0)
        
        if controller.monitoring_thread.is_alive():
            logger.warning("Monitoring thread did not stop gracefully")
        else:
            logger.success("Monitoring thread stopped successfully")
    
    controller.monitoring_active = False
    controller.monitoring_thread = None

# Add methods to controller class
controller.start_monitoring = lambda interval=5.0: start_monitoring(controller, interval)
controller.stop_monitoring = lambda: stop_monitoring(controller)

print("Continuous monitoring system ready")

## 6. Device Connection Configuration

In [ ]:
# Configuration for your devices
# Modify these values according to your setup

PUMP1_COM = 'COM19'    # Adjust to your pump 1 COM port
PUMP1_ADDR = 2         # HiScroll12 standard address is 2

PUMP2_COM = 'COM21'    # Adjust to your pump 2 COM port
PUMP2_ADDR = 2         # HiScroll12 standard address is 2

# TPG366 pressure controller
PRESSURE_COM = 'COM22'  # TPG366 pressure controller COM port
PRESSURE_ADDR = 1       # TPG366 standard address is 1

print("Device configuration loaded:")
print(f"Pump 1: {PUMP1_COM}, Address {PUMP1_ADDR}")
print(f"Pump 2: {PUMP2_COM}, Address {PUMP2_ADDR}")
print(f"TPG366 Pressure Controller: {PRESSURE_COM}, Address {PRESSURE_ADDR}")
print(f"  -> Sensor 1: Channel 1")
print(f"  -> Sensor 2: Channel 2")

In [ ]:
# Connect to HiScroll12 pumps
pump_connection = controller.connect_pumps(PUMP1_COM, PUMP1_ADDR, PUMP2_COM, PUMP2_ADDR)

if pump_connection:
    print("✅ HiScroll12 pumps connected successfully")
else:
    print("❌ Failed to connect to HiScroll12 pumps")

In [ ]:
# Connect to TPG366 pressure controller
pressure_connection = controller.connect_pressure_controller(PRESSURE_COM, PRESSURE_ADDR)

if pressure_connection:
    print("✅ TPG366 pressure controller connected successfully")
    print("   Both sensors accessible via channels 1 and 2")
else:
    print("❌ Failed to connect to TPG366 pressure controller")

## 7. Device Testing and Status

In [ ]:
# Test HiScroll12 Pump 1
if controller.pump1:
    try:
        print("Testing HiScroll12 Pump 1:")
        sw_version = controller.pump1.get_software_version()
        print(f"  Software Version: {sw_version}")
        
        rotation_speed = controller.pump1.get_actual_rotation_speed_rpm()
        print(f"  Rotation Speed: {rotation_speed} RPM")
        
        serial_number = controller.pump1.get_serial_number()
        print(f"  Serial Number: {serial_number}")
        
        pump_enabled = controller.pump1.get_pump_enable()
        print(f"  Pump Enabled: {pump_enabled}")
        
        drive_power = controller.pump1.get_drive_power()
        print(f"  Drive Power: {drive_power} W")
        
        print("  Pump 1 tests completed ✅")
        
    except Exception as e:
        logger.error(f"Error testing Pump 1: {e}")
        print(f"  ❌ Error testing Pump 1: {e}")
else:
    print("Pump 1 not connected")

In [ ]:
# Test HiScroll12 Pump 2
if controller.pump2:
    try:
        print("Testing HiScroll12 Pump 2:")
        sw_version = controller.pump2.get_software_version()
        print(f"  Software Version: {sw_version}")
        
        rotation_speed = controller.pump2.get_actual_rotation_speed_rpm()
        print(f"  Rotation Speed: {rotation_speed} RPM")
        
        serial_number = controller.pump2.get_serial_number()
        print(f"  Serial Number: {serial_number}")
        
        pump_enabled = controller.pump2.get_pump_enable()
        print(f"  Pump Enabled: {pump_enabled}")
        
        drive_power = controller.pump2.get_drive_power()
        print(f"  Drive Power: {drive_power} W")
        
        print("  Pump 2 tests completed ✅")
        
    except Exception as e:
        logger.error(f"Error testing Pump 2: {e}")
        print(f"  ❌ Error testing Pump 2: {e}")
else:
    print("Pump 2 not connected")

In [ ]:
# Test TPG366 pressure controller and sensors
if controller.pressure_controller:
    try:
        print("Testing TPG366 Pressure Controller:")
        
        # Get software version
        sw_version = controller.pressure_controller.get_software_version()
        print(f"  Software Version: {sw_version}")
        
        # Get device name
        device_name = controller.pressure_controller.get_electronics_name()
        print(f"  Device Name: {device_name}")
        
        # Test Sensor 1 (Channel 1)
        print("\n  Testing Sensor 1 (Channel 1):")
        pressure1 = controller.pressure_controller.read_pressure_value(1)
        print(f"    Pressure: {pressure1}")
        
        # Test Sensor 2 (Channel 2)
        print("\n  Testing Sensor 2 (Channel 2):")
        pressure2 = controller.pressure_controller.read_pressure_value(2)
        print(f"    Pressure: {pressure2}")
        
        print("\n  TPG366 pressure controller tests completed successfully ✅")
        
    except Exception as e:
        logger.error(f"Error testing TPG366 pressure controller: {e}")
        print(f"  ❌ Error testing TPG366 pressure controller: {e}")
else:
    print("TPG366 pressure controller not connected")

## 8. Pump Control Interface

In [ ]:
# Pump control functions
def enable_both_pumps():
    """Enable both pumps"""
    try:
        if controller.pump1:
            controller.pump1.enable_pump()
            logger.info("Pump 1 enabled")
        
        if controller.pump2:
            controller.pump2.enable_pump()
            logger.info("Pump 2 enabled")
            
        print("✅ Both pumps enabled")
    except Exception as e:
        logger.error(f"Error enabling pumps: {e}")
        print(f"❌ Error enabling pumps: {e}")

def disable_both_pumps():
    """Disable both pumps"""
    try:
        if controller.pump1:
            controller.pump1.disable_pump()
            logger.info("Pump 1 disabled")
        
        if controller.pump2:
            controller.pump2.disable_pump()
            logger.info("Pump 2 disabled")
            
        print("✅ Both pumps disabled")
    except Exception as e:
        logger.error(f"Error disabling pumps: {e}")
        print(f"❌ Error disabling pumps: {e}")

def enable_pump1():
    """Enable pump 1 only"""
    try:
        if controller.pump1:
            controller.pump1.enable_pump()
            logger.info("Pump 1 enabled")
            print("✅ Pump 1 enabled")
    except Exception as e:
        logger.error(f"Error enabling pump 1: {e}")
        print(f"❌ Error enabling pump 1: {e}")

def enable_pump2():
    """Enable pump 2 only"""
    try:
        if controller.pump2:
            controller.pump2.enable_pump()
            logger.info("Pump 2 enabled")
            print("✅ Pump 2 enabled")
    except Exception as e:
        logger.error(f"Error enabling pump 2: {e}")
        print(f"❌ Error enabling pump 2: {e}")

def disable_pump1():
    """Disable pump 1 only"""
    try:
        if controller.pump1:
            controller.pump1.disable_pump()
            logger.info("Pump 1 disabled")
            print("✅ Pump 1 disabled")
    except Exception as e:
        logger.error(f"Error disabling pump 1: {e}")
        print(f"❌ Error disabling pump 1: {e}")

def disable_pump2():
    """Disable pump 2 only"""
    try:
        if controller.pump2:
            controller.pump2.disable_pump()
            logger.info("Pump 2 disabled")
            print("✅ Pump 2 disabled")
    except Exception as e:
        logger.error(f"Error disabling pump 2: {e}")
        print(f"❌ Error disabling pump 2: {e}")

print("Pump control functions ready")

In [ ]:
# Enable both pumps
enable_both_pumps()

In [ ]:
# Enable only pump 2
enable_pump2()

In [ ]:
# Disable pump 1
disable_pump1()

In [ ]:
# Disable pump 2
disable_pump2()

In [ ]:
# Disable both pumps
disable_both_pumps()

## 9. Monitoring Control Interface

In [ ]:
# Create interactive controls for monitoring
start_button = widgets.Button(description="Start Monitoring", button_style='success')
stop_button = widgets.Button(description="Stop Monitoring", button_style='danger')
interval_slider = widgets.FloatSlider(
    value=5.0,
    min=1.0,
    max=15.0,
    step=1.0,
    description='Interval (s):',
    style={'description_width': 'initial'}
)
status_output = widgets.Output()

def on_start_clicked(b):
    with status_output:
        clear_output(wait=True)
        success = controller.start_monitoring(interval_slider.value)
        if success:
            print(f"✅ Monitoring started with {interval_slider.value}s interval")
        else:
            print("❌ Failed to start monitoring (already running?)")

def on_stop_clicked(b):
    with status_output:
        clear_output(wait=True)
        controller.stop_monitoring()
        print("🛑 Monitoring stopped")

start_button.on_click(on_start_clicked)
stop_button.on_click(on_stop_clicked)

# Display controls
controls = widgets.VBox([
    widgets.Label("Monitoring Controls:"),
    interval_slider,
    widgets.HBox([start_button, stop_button]),
    status_output
])

display(controls)

## 10. Data Visualization

### 10.1 Static Plotting

In [ ]:
def plot_monitoring_data():
    """Plot the collected monitoring data"""
    if not controller.monitoring_data:
        print("No monitoring data available. Start monitoring first.")
        return
    
    # Convert data to lists for easier plotting
    timestamps = []
    pump1_rpm = []
    pump2_rpm = []
    sensor1_pressure = []
    sensor2_pressure = []
    pump1_power = []
    pump2_power = []
    pump1_temp_motor = []
    pump2_temp_motor = []
    
    for cycle in controller.monitoring_data:
        timestamps.append(cycle['cycle_timestamp'])
        
        # Extract pump data
        pump1_rpm.append(cycle['pump1_data']['rotation_speed_rpm'] if cycle['pump1_data'] and cycle['pump1_data'].get('rotation_speed_rpm') else None)
        pump2_rpm.append(cycle['pump2_data']['rotation_speed_rpm'] if cycle['pump2_data'] and cycle['pump2_data'].get('rotation_speed_rpm') else None)
        pump1_power.append(cycle['pump1_data']['drive_power'] if cycle['pump1_data'] and cycle['pump1_data'].get('drive_power') else None)
        pump2_power.append(cycle['pump2_data']['drive_power'] if cycle['pump2_data'] and cycle['pump2_data'].get('drive_power') else None)
        pump1_temp_motor.append(cycle['pump1_data']['temp_motor'] if cycle['pump1_data'] and cycle['pump1_data'].get('temp_motor') else None)
        pump2_temp_motor.append(cycle['pump2_data']['temp_motor'] if cycle['pump2_data'] and cycle['pump2_data'].get('temp_motor') else None)
        
        # Extract pressure data
        sensor1_pressure.append(cycle['sensor1_data']['pressure_value'] if cycle['sensor1_data'] and cycle['sensor1_data'].get('pressure_value') else None)
        sensor2_pressure.append(cycle['sensor2_data']['pressure_value'] if cycle['sensor2_data'] and cycle['sensor2_data'].get('pressure_value') else None)
    
    # Create subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 10))
    
    # Plot pump RPM
    ax1.plot(timestamps, pump1_rpm, 'b-', label='Pump 1 RPM', marker='o')
    ax1.plot(timestamps, pump2_rpm, 'r-', label='Pump 2 RPM', marker='s')
    ax1.set_ylabel('Rotation Speed (RPM)')
    ax1.set_title('Pump Rotation Speed Over Time')
    ax1.legend()
    ax1.grid(True)
    
    # Plot pressure (with logarithmic scale for vacuum)
    ax2.plot(timestamps, sensor1_pressure, 'b-', label='Sensor 1 Pressure', marker='^')
    ax2.plot(timestamps, sensor2_pressure, 'r-', label='Sensor 2 Pressure', marker='v')
    ax2.set_yscale('log')
    ax2.set_ylabel('Pressure (log scale)')
    ax2.set_title('Pressure Over Time')
    ax2.legend()
    ax2.grid(True)
    
    # Plot pump power
    ax3.plot(timestamps, pump1_power, 'b-', label='Pump 1 Power', marker='o')
    ax3.plot(timestamps, pump2_power, 'r-', label='Pump 2 Power', marker='s')
    ax3.set_ylabel('Power (W)')
    ax3.set_title('Pump Power Over Time')
    ax3.legend()
    ax3.grid(True)
    
    # Plot motor temperature
    ax4.plot(timestamps, pump1_temp_motor, 'b-', label='Pump 1 Motor Temp', marker='o')
    ax4.plot(timestamps, pump2_temp_motor, 'r-', label='Pump 2 Motor Temp', marker='s')
    ax4.set_ylabel('Temperature (°C)')
    ax4.set_xlabel('Time')
    ax4.set_title('Motor Temperature Over Time')
    ax4.legend()
    ax4.grid(True)
    
    # Rotate x-axis labels for better readability
    for ax in [ax1, ax2, ax3, ax4]:
        ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"Plotted {len(controller.monitoring_data)} data points")
    valid_p1 = [p for p in sensor1_pressure if p is not None]
    valid_p2 = [p for p in sensor2_pressure if p is not None]
    
    if valid_p1:
        print(f"Sensor 1 pressure range: {min(valid_p1):.2e} to {max(valid_p1):.2e}")
    if valid_p2:
        print(f"Sensor 2 pressure range: {min(valid_p2):.2e} to {max(valid_p2):.2e}")

def get_monitoring_summary():
    """Get a summary of current monitoring status"""
    print(f"Monitoring Status: {'🟢 Active' if controller.monitoring_active else '🔴 Inactive'}")
    print(f"Data Points Collected: {len(controller.monitoring_data)}")
    print(f"Pump 1 Connected: {'✅' if controller.pump1 else '❌'}")
    print(f"Pump 2 Connected: {'✅' if controller.pump2 else '❌'}")
    print(f"TPG366 Controller Connected: {'✅' if controller.pressure_controller else '❌'}")
    
    if controller.monitoring_data:
        latest = controller.monitoring_data[-1]
        print(f"\nLatest Data Point: {latest['cycle_timestamp']}")
        
        # Show latest pressure readings
        if latest['sensor1_data']:
            p1 = latest['sensor1_data'].get('pressure_value')
            print(f"  Sensor 1 Pressure: {p1}")
        
        if latest['sensor2_data']:
            p2 = latest['sensor2_data'].get('pressure_value') 
            print(f"  Sensor 2 Pressure: {p2}")

print("Data visualization functions ready")

In [ ]:
# Plot monitoring data (run this cell periodically to see updated plots)
plot_monitoring_data()

In [ ]:
# Get current monitoring summary
get_monitoring_summary()

### 10.2 Live Plotting (Same as original - no changes to logic)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

class PumpLockerPlotter:
    def __init__(self, controller, update_interval=1000):
        """
        Initialize live monitoring plot
        Args:
            controller: The controller object with monitoring_data
            update_interval: Update interval in milliseconds (default: 1000ms = 1 second)
        """
        self.controller = controller
        self.update_interval = update_interval

        # Initialize data lists
        self.timestamps = []
        self.pump1_rpm = []
        self.pump2_rpm = []
        self.sensor1_pressure = []
        self.sensor2_pressure = []
        self.pump1_power = []
        self.pump2_power = []
        self.pump1_temp_motor = []
        self.pump1_temp_pwrstg = []
        self.pump2_temp_motor = []
        self.pump2_temp_pwrstg = []

        # Create figure and subplots
        self.fig, ((self.ax1, self.ax2), (self.ax3, self.ax4)) = plt.subplots(2, 2, figsize=(16, 8))

        self.temp_log = []
        self.log_file_size = 0

        # Animation object
        self.animation = None

    def update_plot(self, frame):
        # Get new data
        current_size = len(self.controller.monitoring_data)
        if current_size > self.log_file_size:
            new_entries = self.controller.monitoring_data[self.log_file_size:]
            self.temp_log.extend(new_entries)
            self.log_file_size = current_size

        # Process new entries
        for entry in self.temp_log:
            self.timestamps.append(entry['cycle_timestamp'])
            
            # Extract pump data
            self.pump1_rpm.append(entry['pump1_data']['rotation_speed_rpm'] if entry['pump1_data'] else None)
            self.pump2_rpm.append(entry['pump2_data']['rotation_speed_rpm'] if entry['pump2_data'] else None)
            self.sensor1_pressure.append(entry['sensor1_data']['pressure_value'] if entry['sensor1_data'] else None)
            self.sensor2_pressure.append(entry['sensor2_data']['pressure_value'] if entry['sensor2_data'] else None)
            self.pump1_power.append(entry['pump1_data']['drive_power'] if entry['pump1_data'] else None)
            self.pump2_power.append(entry['pump2_data']['drive_power'] if entry['pump2_data'] else None)
            self.pump1_temp_motor.append(entry['pump1_data']['temp_motor'] if entry['pump1_data'] else None)
            self.pump1_temp_pwrstg.append(entry['pump1_data']['temp_power_stage'] if entry['pump1_data'] else None)
            self.pump2_temp_motor.append(entry['pump2_data']['temp_motor'] if entry['pump2_data'] else None)
            self.pump2_temp_pwrstg.append(entry['pump2_data']['temp_power_stage'] if entry['pump2_data'] else None)

        # Clear temp_log after processing to avoid reprocessing
        self.temp_log = []

        # Clear only the axes, not the entire figure
        self.ax1.clear()
        self.ax2.clear()
        self.ax3.clear()
        self.ax4.clear()

        # Filter out None values for plotting
        valid_data1 = [(t, p1, p2) for t, p1, p2 in zip(self.timestamps, self.pump1_rpm, self.pump2_rpm)
                      if p1 is not None or p2 is not None]
        valid_data2 = [(t, s1, s2) for t, s1, s2 in zip(self.timestamps, self.sensor1_pressure, self.sensor2_pressure)
                      if s1 is not None or s2 is not None]
        valid_data3 = [(t, pw1, pw2) for t, pw1, pw2 in zip(self.timestamps, self.pump1_power, self.pump2_power)
                      if pw1 is not None or pw2 is not None]
        valid_data4 = [(t, tm1, tp1, tm2, tp2) for t, tm1, tp1, tm2, tp2 in zip(self.timestamps, self.pump1_temp_motor,
                      self.pump1_temp_pwrstg, self.pump2_temp_motor, self.pump2_temp_pwrstg)
                      if tm1 is not None or tp1 is not None or tm2 is not None or tp2 is not None]

        # Plot RPM data (ax1) - keep only last 120 points for performance
        if valid_data1:
            times1, pump1_vals, pump2_vals = zip(*valid_data1)

            # Plot pump1 data (filter None values) - BLUE with circle marker
            pump1_valid = [(t, p) for t, p in zip(times1, pump1_vals) if p is not None]
            if pump1_valid:
                t1, p1 = zip(*pump1_valid)
                if len(t1) > 120:
                    self.ax1.plot(t1[-119:], p1[-119:], 'b-', label='Pump 1 RPM', marker='o')
                else:
                    self.ax1.plot(t1, p1, 'b-', label='Pump 1 RPM', marker='o')

            # Plot pump2 data (filter None values) - RED with square marker
            pump2_valid = [(t, p) for t, p in zip(times1, pump2_vals) if p is not None]
            if pump2_valid:
                t2, p2 = zip(*pump2_valid)
                if len(t2) > 120:
                    self.ax1.plot(t2[-119:], p2[-119:], 'r-', label='Pump 2 RPM', marker='s')
                else:
                    self.ax1.plot(t2, p2, 'r-', label='Pump 2 RPM', marker='s')

        # Plot pressure data (ax2)
        if valid_data2:
            times2, sensor1_vals, sensor2_vals = zip(*valid_data2)

            # Plot sensor1 data (filter None values) - BLUE with circle marker (matches pump1)
            sensor1_valid = [(t, s) for t, s in zip(times2, sensor1_vals) if s is not None]
            if sensor1_valid:
                t1, s1 = zip(*sensor1_valid)
                if len(t1) > 120:
                    self.ax2.plot(t1[-119:], s1[-119:], 'b-', label='Sensor 1 Pressure', marker='o')
                else:
                    self.ax2.plot(t1, s1, 'b-', label='Sensor 1 Pressure', marker='o')

            # Plot sensor2 data (filter None values) - RED with square marker (matches pump2)
            sensor2_valid = [(t, s) for t, s in zip(times2, sensor2_vals) if s is not None]
            if sensor2_valid:
                t2, s2 = zip(*sensor2_valid)
                if len(t2) > 120:
                    self.ax2.plot(t2[-119:], s2[-119:], 'r-', label='Sensor 2 Pressure', marker='s')
                else:
                    self.ax2.plot(t2, s2, 'r-', label='Sensor 2 Pressure', marker='s')

        # Plot power data (ax3)
        if valid_data3:
            times3, power1_vals, power2_vals = zip(*valid_data3)

            # Plot pump1 power data (filter None values) - BLUE with circle marker
            power1_valid = [(t, p) for t, p in zip(times3, power1_vals) if p is not None]
            if power1_valid:
                t1, p1 = zip(*power1_valid)
                if len(t1) > 120:
                    self.ax3.plot(t1[-119:], p1[-119:], 'b-', label='Pump 1 Power', marker='o')
                else:
                    self.ax3.plot(t1, p1, 'b-', label='Pump 1 Power', marker='o')

            # Plot pump2 power data (filter None values) - RED with square marker
            power2_valid = [(t, p) for t, p in zip(times3, power2_vals) if p is not None]
            if power2_valid:
                t2, p2 = zip(*power2_valid)
                if len(t2) > 120:
                    self.ax3.plot(t2[-119:], p2[-119:], 'r-', label='Pump 2 Power', marker='s')
                else:
                    self.ax3.plot(t2, p2, 'r-', label='Pump 2 Power', marker='s')

        # Plot temperature data (ax4)
        if valid_data4:
            times4, temp_motor1_vals, temp_pwrstg1_vals, temp_motor2_vals, temp_pwrstg2_vals = zip(*valid_data4)

            # Plot pump1 motor temperature data (filter None values) - BLUE with circle marker
            temp_motor1_valid = [(t, tm) for t, tm in zip(times4, temp_motor1_vals) if tm is not None]
            if temp_motor1_valid:
                t1, tm1 = zip(*temp_motor1_valid)
                if len(t1) > 120:
                    self.ax4.plot(t1[-119:], tm1[-119:], 'b-', label='Pump 1 Motor Temp', marker='o')
                else:
                    self.ax4.plot(t1, tm1, 'b-', label='Pump 1 Motor Temp', marker='o')

            # Plot pump1 power stage temperature data (filter None values) - CYAN with triangle marker
            temp_pwrstg1_valid = [(t, tp) for t, tp in zip(times4, temp_pwrstg1_vals) if tp is not None]
            if temp_pwrstg1_valid:
                t1, tp1 = zip(*temp_pwrstg1_valid)
                if len(t1) > 120:
                    self.ax4.plot(t1[-119:], tp1[-119:], 'c-', label='Pump 1 Power Stage Temp', marker='^')
                else:
                    self.ax4.plot(t1, tp1, 'c-', label='Pump 1 Power Stage Temp', marker='^')

            # Plot pump2 motor temperature data (filter None values) - RED with square marker
            temp_motor2_valid = [(t, tm) for t, tm in zip(times4, temp_motor2_vals) if tm is not None]
            if temp_motor2_valid:
                t2, tm2 = zip(*temp_motor2_valid)
                if len(t2) > 120:
                    self.ax4.plot(t2[-119:], tm2[-119:], 'r-', label='Pump 2 Motor Temp', marker='s')
                else:
                    self.ax4.plot(t2, tm2, 'r-', label='Pump 2 Motor Temp', marker='s')

            # Plot pump2 power stage temperature data (filter None values) - MAGENTA with inverted triangle marker
            temp_pwrstg2_valid = [(t, tp) for t, tp in zip(times4, temp_pwrstg2_vals) if tp is not None]
            if temp_pwrstg2_valid:
                t2, tp2 = zip(*temp_pwrstg2_valid)
                if len(t2) > 120:
                    self.ax4.plot(t2[-119:], tp2[-119:], 'm-', label='Pump 2 Power Stage Temp', marker='v')
                else:
                    self.ax4.plot(t2, tp2, 'm-', label='Pump 2 Power Stage Temp', marker='v')

        # Set labels and formatting
        self.ax1.set_ylabel('Rotation Speed (RPM)')
        self.ax1.set_title('Pump Rotation Speed Over Time')
        self.ax1.legend()
        self.ax1.grid(True)

        # Use logarithmic scale for pressure if values are very small (typical for vacuum)
        self.ax2.set_yscale('log')
        self.ax2.set_ylabel('Pressure (log scale)')
        self.ax2.set_title('Pressure Over Time')
        self.ax2.legend()
        self.ax2.grid(True)

        self.ax3.set_ylabel('Power Consumption')
        self.ax3.set_title('Pump Power Over Time')
        self.ax3.legend()
        self.ax3.grid(True)

        self.ax4.set_ylabel('Temperature')
        self.ax4.set_xlabel('Time')
        self.ax4.set_title('Motor and Power Stage Temperature Over Time')
        self.ax4.legend()
        self.ax4.grid(True)

        # Rotate x-axis labels for better readability
        for ax in [self.ax1, self.ax2, self.ax3, self.ax4]:
            ax.tick_params(axis='x', rotation=45)

        # Adjust layout
        self.fig.tight_layout()

    def start_live_plot(self):
        """Start the live plotting"""
        if not self.controller.monitoring_data:
            print("No monitoring data available. Start monitoring first.")
            return None, None

        print(f"Starting live plot with {self.update_interval/1000}s update interval...")
        print("Close the plot window to stop live plotting.")

        # Create animation
        self.ani = FuncAnimation(
            self.fig,
            self.update_plot,
            interval=self.update_interval,
            blit=False,  # Set to False for easier debugging
            cache_frame_data=False
        )

        # Show the plot
        plt.show()

        return self.timestamps, self.controller.monitoring_data

    def stop_live_plot(self):
        """Stop the live plotting"""
        if hasattr(self, 'ani'):
            self.ani.event_source.stop()
            print("Live plotting stopped.")
        else:
            print("No animation to stop.")

print("Live plotting class ready")

In [ ]:
%matplotlib notebook

In [ ]:
# Start live plotting
live_plot = PumpLockerPlotter(controller)
timestamps, data = live_plot.start_live_plot()

In [ ]:
# Stop live plotting
live_plot.stop_live_plot()

## 11. Data Export and Cleanup

In [ ]:
def export_monitoring_data(filename: str = None):
    """Export monitoring data to CSV file"""
    if not controller.monitoring_data:
        print("No data to export")
        return
    
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"pump_locker_data_{timestamp}.csv"
    
    # Flatten the data for CSV export
    flattened_data = []
    for cycle in controller.monitoring_data:
        row = {'timestamp': cycle['cycle_timestamp']}
        
        # Add pump 1 data
        if cycle['pump1_data']:
            for key, value in cycle['pump1_data'].items():
                row[f'pump1_{key}'] = value
        
        # Add pump 2 data
        if cycle['pump2_data']:
            for key, value in cycle['pump2_data'].items():
                row[f'pump2_{key}'] = value
        
        # Add sensor 1 data
        if cycle['sensor1_data']:
            for key, value in cycle['sensor1_data'].items():
                row[f'sensor1_{key}'] = value
        
        # Add sensor 2 data
        if cycle['sensor2_data']:
            for key, value in cycle['sensor2_data'].items():
                row[f'sensor2_{key}'] = value
        
        flattened_data.append(row)
    
    df = pd.DataFrame(flattened_data)
    df.to_csv(filename, index=False)
    logger.info(f"Data exported to {filename}")
    print(f"✅ Data exported to {filename}")

def save_monitoring_data_pickle(filename: str = None):
    """Save monitoring data as pickle file"""
    if not controller.monitoring_data:
        print("No data to save")
        return
        
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"pump_locker_data_{timestamp}.pkl"
    
    with open(filename, 'wb') as f:
        pickle.dump(controller.monitoring_data, f)
    
    logger.info(f"Data saved to pickle file: {filename}")
    print(f"✅ Data saved to pickle file: {filename}")

def load_monitoring_data_pickle(filename: str):
    """Load monitoring data from pickle file"""
    try:
        with open(filename, 'rb') as f:
            loaded_data = pickle.load(f)
        
        controller.monitoring_data = loaded_data
        logger.info(f"Data loaded from pickle file: {filename}")
        print(f"✅ Data loaded from pickle file: {filename}")
        print(f"Loaded {len(loaded_data)} data points")
        
    except Exception as e:
        logger.error(f"Error loading data from {filename}: {e}")
        print(f"❌ Error loading data from {filename}: {e}")

print("Data export functions ready")

In [ ]:
# Export current monitoring data
timestamp = datetime.now().strftime('%d-%m-%Y_%H%M')
export_monitoring_data(f"{timestamp}_pump_locker_data.csv")

In [ ]:
# Save monitoring data as pickle
timestamp = datetime.now().strftime('%d-%m-%Y_%H%M')
save_monitoring_data_pickle(f"{timestamp}_pump_locker_data.pkl")

In [ ]:
# Emergency disconnect all devices
controller.disconnect_all()

In [ ]:
# Emergency stop and cleanup
print("🛑 Emergency Stop - Disconnecting all devices...")
controller.disconnect_all()
print("✅ All devices disconnected safely")
print("\nTo reconnect, run the connection cells again.")

## Quick Reference

### Key Functions:
- **Connect devices**: Run cells in section 6
- **Start monitoring**: Use the interface in section 9 or call `controller.start_monitoring(interval)`
- **Stop monitoring**: Use the interface in section 9 or call `controller.stop_monitoring()`
- **Control pumps**: Use functions in section 8 (enable/disable individual or both pumps)
- **View data**: Run `get_monitoring_summary()` and `plot_monitoring_data()`
- **Export data**: Run `export_monitoring_data()` or `save_monitoring_data_pickle()`
- **Emergency stop**: Run the last cell to disconnect everything

### Key Improvements from Original:
- Uses current hardware APIs (HiScroll12 and TPG366 classes)
- Shared logging system with loguru for all devices
- Simplified device initialization with shared logger and thread lock
- Updated monitoring functions to use current device methods
- Live plotting logic kept unchanged as requested
- Proper error handling and status reporting
- Support for both CSV and pickle data export

### Device Configuration:
- Pump 1 & 2: HiScroll12 pumps (standard address 2)
- Pressure Controller: TPG366 (standard address 1, channels 1 & 2)
- All devices use shared logger and thread synchronization